In [1]:
import pandas as pd
from pandas import Series,DataFrame
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline
from sklearn.metrics import roc_auc_score, auc

In [2]:
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from operator import itemgetter

In [4]:
act_train_df = pd.read_csv('./red-hat_data/act_train.csv', dtype={'people_id': np.str, 'activity_id': np.str, 'outcome': np.int8}, parse_dates=['date'])
act_test_df = pd.read_csv('./red-hat_data/act_test.csv', dtype={'people_id': np.str, 'activity_id': np.str}, parse_dates=['date'])
ppl_df = pd.read_csv('./red-hat_data/people.csv', dtype={'people_id': np.str, 'activity_id': np.str, 'char_38': np.int32}, parse_dates=['date'])

In [5]:
test_id = act_test_df.activity_id

In [6]:
train_df = pd.read_csv('./processing_data/train.csv')
test_df = pd.read_csv('./processing_data/test.csv', )

In [9]:
scaler = MinMaxScaler()

In [19]:
for df in [train_df, test_df]:
    df['char_38'] = scaler.fit_transform(np.array(df['char_38']).reshape(-1,1))

In [22]:
X_train = train_df.drop(['outcome'], axis=1)
Y_train = train_df['outcome']

In [23]:
whole_df = pd.concat([X_train,test_df],ignore_index=True,sort=False)

In [26]:
objective_var=['group_1','activity_category','char_1_x','char_2_x','char_3_x','char_4_x','char_5_x','char_6_x','char_7_x','char_8_x','char_9_x','char_10_x','char_1_y','char_2_y','char_3_y','char_4_y','char_5_y','char_6_y','char_7_y','char_8_y','char_9_y']

In [24]:
def reduce_dim(df, col, toreplace):
    for index,i in df[col].duplicated(keep=False).iteritems():
        if i==False:
            df.set_value(index,col,toreplace)
    return df

In [27]:
for var in objective_var:
    X_whole = reduce_dim(whole_df, var, 9999999)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  after removing the cwd from sys.path.


In [28]:
X = X_whole[:len(train_df)]
X_test = X_whole[len(train_df):]

In [29]:
del act_train_df, act_test_df, ppl_df
del train_df, test_df
del whole_df, X_whole

In [54]:
trainval = list(X.columns.values)
testval = list(X_test.columns.values)
features = sorted(list(set(trainval) & set(testval)))
features

['activity_category',
 'activity_id',
 'char_10_x',
 'char_10_y',
 'char_11',
 'char_12',
 'char_13',
 'char_14',
 'char_15',
 'char_16',
 'char_17',
 'char_18',
 'char_19',
 'char_1_x',
 'char_1_y',
 'char_20',
 'char_21',
 'char_22',
 'char_23',
 'char_24',
 'char_25',
 'char_26',
 'char_27',
 'char_28',
 'char_29',
 'char_2_x',
 'char_2_y',
 'char_30',
 'char_31',
 'char_32',
 'char_33',
 'char_34',
 'char_35',
 'char_36',
 'char_37',
 'char_38',
 'char_3_x',
 'char_3_y',
 'char_4_x',
 'char_4_y',
 'char_5_x',
 'char_5_y',
 'char_6_x',
 'char_6_y',
 'char_7_x',
 'char_7_y',
 'char_8_x',
 'char_8_y',
 'char_9_x',
 'char_9_y',
 'day_x',
 'day_y',
 'group_1',
 'isweekend_x',
 'isweekend_y',
 'month_x',
 'month_y',
 'year_x',
 'year_y']

In [56]:
def create_feature_map(features):
    outfile = open('./xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
    outfile.close()

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, Y_train, test_size = 0.5, random_state=1)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

In [30]:
le = LabelEncoder()
enc = OneHotEncoder(handle_unknown='ignore')

In [31]:
not_object = []
for var in X.columns:
    if var not in objective_var:
        not_object.append(var)
    else:
        temp = pd.concat([X[var],X_test[var]], sort=False)        
        le.fit(temp.values)
        X[var] = le.transform(X[var].values)
        X_test[var] = le.transform(X_test[var].values) 

In [32]:
enc = enc.fit(pd.concat([X[objective_var],X_test[objective_var]]))
X_obj_sparse = enc.transform(X[objective_var])
X_test_obj_sparse = enc.transform(X_test[objective_var])

In [33]:
X[not_object].shape, X[objective_var].shape, X_test[not_object].shape, X_test[objective_var].shape

((2197291, 38), (2197291, 21), (498687, 38), (498687, 21))

In [34]:
from scipy.sparse import hstack

X_sparse = hstack((X[not_object], X_obj_sparse))
X_test_sparse = hstack((X_test[not_object], X_test_obj_sparse))

In [35]:
X_sparse.shape, X_test_sparse.shape

((2197291, 37439), (498687, 37439))

In [36]:
dtrain = xgb.DMatrix(X_sparse,label=Y_train)
dtest = xgb.DMatrix(X_test_sparse)

/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/opt/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


In [49]:
param = {'max_depth':10, 'eta':0.02, 'silent':1, 'objective':'binary:logistic' }
param['nthread'] = 4
param['eval_metric'] = 'auc'
param['subsample'] = 0.5
param['colsample_bytree']= 0.7
param['min_child_weight'] = 2
param['booster'] = "gblinear"

In [50]:
# np.random.seed(120)
evals  = [(dtrain,'train')]
num_round = 300
bst = xgb.train(param, dtrain, num_round, evals, early_stopping_rounds=10, verbose_eval=10)

[0]	train-auc:0.890781
Will train until train-auc hasn't improved in 10 rounds.
[10]	train-auc:0.96075
[20]	train-auc:0.987145
[30]	train-auc:0.992606
[40]	train-auc:0.994636
[50]	train-auc:0.995679
[60]	train-auc:0.996289
[70]	train-auc:0.996669
[80]	train-auc:0.996915
[90]	train-auc:0.997079
[100]	train-auc:0.997189
[110]	train-auc:0.997266
[120]	train-auc:0.997322
[130]	train-auc:0.997364
[140]	train-auc:0.997398
[150]	train-auc:0.997426
[160]	train-auc:0.997449
[170]	train-auc:0.997467
[180]	train-auc:0.997482
[190]	train-auc:0.997495
[200]	train-auc:0.997506
[210]	train-auc:0.997516
[220]	train-auc:0.997525
[230]	train-auc:0.997532
[240]	train-auc:0.997539
[250]	train-auc:0.997545
[260]	train-auc:0.99755
[270]	train-auc:0.997555
[280]	train-auc:0.99756
[290]	train-auc:0.997564
[299]	train-auc:0.997567


In [60]:
param1 = {'max_depth':10, 'eta':0.02, 'silent':1, 'objective':'binary:logistic' }
param1['nthread'] = 4
param1['eval_metric'] = 'auc'
param1['subsample'] = 0.5
param1['colsample_bytree']= 0.7
param1['min_child_weight'] = 2
param1['booster'] = "gbtree"

In [61]:
np.random.seed(120)
evals  = [(dtrain,'train')]
num_round = 300
bst1 = xgb.train(param1, dtrain, num_round, evals, early_stopping_rounds=10, verbose_eval=10)

[0]	train-auc:0.906377
Will train until train-auc hasn't improved in 10 rounds.
[10]	train-auc:0.931054
[20]	train-auc:0.933289
[30]	train-auc:0.934578
[40]	train-auc:0.935977
[50]	train-auc:0.936964
[60]	train-auc:0.937954
[70]	train-auc:0.939348
[80]	train-auc:0.940233
[90]	train-auc:0.941185
[100]	train-auc:0.941935
[110]	train-auc:0.943195
[120]	train-auc:0.943929
[130]	train-auc:0.944833
[140]	train-auc:0.945781
[150]	train-auc:0.946413
[160]	train-auc:0.946996
[170]	train-auc:0.947631
[180]	train-auc:0.947975
[190]	train-auc:0.948456
[200]	train-auc:0.948721
[210]	train-auc:0.949081
[220]	train-auc:0.949374
[230]	train-auc:0.949656
[240]	train-auc:0.95004
[250]	train-auc:0.950291
[260]	train-auc:0.950598
[270]	train-auc:0.95093
[280]	train-auc:0.951127
[290]	train-auc:0.951326
[299]	train-auc:0.951533


In [63]:
def get_importance(gbm, features):
    create_feature_map(features)
    importance = gbm.get_fscore(fmap='xgb.fmap')
    importance = sorted(importance.items(), key=itemgetter(1), reverse=True)
    return importance

In [66]:
imp = get_importance(bst1, features)
imp

[('char_36', 1232),
 ('char_10_x', 1112),
 ('char_3_x', 737),
 ('activity_id', 655),
 ('char_10_y', 525),
 ('char_38', 486),
 ('char_37', 299),
 ('f11028', 256),
 ('f37391', 195),
 ('f37410', 186),
 ('f30969', 167),
 ('f37395', 160),
 ('char_12', 160),
 ('f18490', 153),
 ('char_25', 153),
 ('f37390', 149),
 ('f37308', 138),
 ('f37424', 115),
 ('f37397', 110),
 ('char_30', 99),
 ('char_29', 97),
 ('f37423', 96),
 ('f37392', 95),
 ('f37412', 89),
 ('f30972', 89),
 ('f614', 89),
 ('char_15', 84),
 ('f37404', 84),
 ('f37401', 83),
 ('char_16', 83),
 ('char_1_x', 80),
 ('char_3_y', 80),
 ('f37432', 77),
 ('f37431', 76),
 ('char_32', 76),
 ('char_20', 76),
 ('char_14', 76),
 ('f37362', 74),
 ('char_24', 73),
 ('char_2_y', 71),
 ('f37409', 71),
 ('char_18', 70),
 ('f31165', 70),
 ('f16157', 70),
 ('char_21', 70),
 ('f37316', 68),
 ('f37388', 68),
 ('f2740', 68),
 ('f37311', 68),
 ('char_17', 68),
 ('f11436', 67),
 ('f37433', 67),
 ('char_22', 67),
 ('f37402', 66),
 ('char_13', 66),
 ('f37317'

In [ ]:
if label_cv == 1 : # maker sure the cv folds don't share the same labels        
    kf = LabelKFold(labels, n_folds=5)
else:  
    kf = StratifiedKFold(Y_train, n_folds=nfolds, shuffle=True, random_state=random_state)

In [71]:
XGB = XGBClassifier()
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [72]:
param_test1 = {
    'n_estimators':range(5,100,10),
    'learning_rate':[0.01,0.1,0.2]
    }

In [79]:
gsearch = GridSearchCV(estimator = XGB,param_grid = param_test1, 
        scoring='roc_auc', n_jobs = -1, iid=False, verbose=3, 
        cv=kfold)

In [86]:
gsearch.fit(X_sparse, Y_train) 

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


KeyboardInterrupt: 

In [87]:
gsearch.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=95, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [88]:
gsearch.best_params_

{'learning_rate': 0.2, 'n_estimators': 95}

In [89]:
gsearch.best_score_

0.9334227088677759

In [95]:
clf = XGBClassifier(n_estimators=95, learning_rate=0.2)
clf.fit(X_sparse, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=95, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [96]:
y_pred_clf = clf.predict(X_test_sparse)

In [97]:
submission9 = pd.DataFrame({'activity_id' : test_id, 'outcome': y_pred_clf})
submission9.to_csv('submission9.csv', index = False)

In [ ]:
# 0.86173

In [98]:
XGB1 = XGB.set_params(**gsearch.best_params_)

In [99]:
param_test2 = {
#     'n_estimators':range(5,100,10),
#     'learning_rate':[0.01,0.1,0.2],
    'max_depth': np.linspace(1, 10, 10, dtype=int)
    }

In [100]:
gsearch1 = GridSearchCV(estimator = XGB1, param_grid = param_test2, 
        scoring='roc_auc', n_jobs = -1, iid=False, verbose=3, 
        cv=kfold)

In [101]:
gsearch1.fit(X_sparse, Y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 21.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 94.3min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.2, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=95, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid=False, n_jobs=-1,
             param_grid={'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
             p

In [102]:
gsearch1.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=95, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [103]:
gsearch1.best_params_

{'max_depth': 10}

In [104]:
gsearch1.best_score_

0.9632642865279267

In [105]:
clf1 = XGBClassifier(n_estimators=95, learning_rate=0.2, max_depth=10)
clf1.fit(X_sparse, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=10,
              min_child_weight=1, missing=None, n_estimators=95, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [111]:
y_pred_clf1 = clf1.predict(X_test_sparse)

In [112]:
submission10 = pd.DataFrame({'activity_id' : test_id, 'outcome': y_pred_clf1})
submission10.to_csv('submission10.csv', index = False)

In [ ]:
# 0.87391

In [106]:
XGB2 = XGB1.set_params(**gsearch1.best_params_)

In [107]:
param_test3 = {
    'min_child_weight':range(1,6,2),
    'max_depth': range(2,10,1)
}

In [108]:
gsearch2 = GridSearchCV(estimator = XGB2, param_grid = param_test3, 
        scoring='roc_auc', n_jobs = -1, iid=False, verbose=3, 
        cv=kfold)

In [109]:
gsearch2.fit(X_sparse, Y_train)

Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 20.6min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed: 211.2min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.2, max_delta_step=0,
                                     max_depth=10, min_child_weight=1,
                                     missing=None, n_estimators=95, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid=False, n_jobs=-1,
             param_grid={'max_depth': range(2, 10),
                         'min_child_weight': rang

In [113]:
gsearch2.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=9,
              min_child_weight=3, missing=None, n_estimators=95, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [114]:
gsearch2.best_params_

{'max_depth': 9, 'min_child_weight': 3}

In [115]:
gsearch2.best_score_

0.9597520044821763

In [116]:
clf2 = XGBClassifier(n_estimators=95, learning_rate=0.2, max_depth=9, min_child_weight=3)
clf2.fit(X_sparse, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=9,
              min_child_weight=3, missing=None, n_estimators=95, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [117]:
y_pred_clf2 = clf2.predict(X_test_sparse)

In [118]:
submission11 = pd.DataFrame({'activity_id' : test_id, 'outcome': y_pred_clf2})
submission11.to_csv('submission11.csv', index = False)

In [ ]:
# 0.87174

In [119]:
XGB3 = XGB2.set_params(**gsearch2.best_params_)

In [121]:
XGB3

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.2, max_delta_step=0, max_depth=9,
              min_child_weight=3, missing=None, n_estimators=95, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [120]:
param_test4 = {
#     'min_child_weight':range(1,6,2),
#     'max_depth': range(2,10,1),
    'gamma':[i/10.0 for i in range(0,5)]
}

In [122]:
gsearch3 = GridSearchCV(estimator = XGB3, param_grid = param_test4, 
        scoring='roc_auc', n_jobs = -1, iid=False, verbose=3, 
        cv=kfold)

In [123]:
gsearch3.fit(X_sparse, Y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  25 | elapsed: 63.7min remaining: 20.1min
[Parallel(n_jobs=-1)]: Done  25 out of  25 | elapsed: 73.1min finished


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=1, shuffle=True),
             error_score='raise-deprecating',
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.2, max_delta_step=0,
                                     max_depth=9, min_child_weight=3,
                                     missing=None, n_estimators=95, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid=False, n_jobs=-1,
             param_grid={'gamma': [0.0, 0.1, 0.2, 0.3, 0.4]},
             pre_dispatch='2*n_jobs', re

In [124]:
gsearch3.best_estimator_

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.3,
              learning_rate=0.2, max_delta_step=0, max_depth=9,
              min_child_weight=3, missing=None, n_estimators=95, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [125]:
gsearch3.best_params_

{'gamma': 0.3}

In [126]:
gsearch3.best_score_

0.9599265396963835

In [127]:
clf3 = XGBClassifier(n_estimators=95, learning_rate=0.2, max_depth=9, min_child_weight=3,gamma=0.3)
clf3.fit(X_sparse, Y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.3,
              learning_rate=0.2, max_delta_step=0, max_depth=9,
              min_child_weight=3, missing=None, n_estimators=95, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [128]:
y_pred_ = clf3.predict(X_test_sparse)

In [129]:
submission12 = pd.DataFrame({'activity_id' : test_id, 'outcome': y_pred_clf3})
submission12.to_csv('submission12.csv', index = False)

In [ ]:
# 0.87174